In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sklearn.metrics import roc_curve

In [ ]:
from sklearn.metrics import auc

In [ ]:
df =pd.read_csv("/kaggle/input/bank-loan-classification/UniversalBank.csv")

In [ ]:
df.head(9)

In [ ]:
nulls=df.isnull().sum()
nulls

In [ ]:
#no null values

In [ ]:
nulls_percentage = nulls[nulls!=0]/df.shape[0]*100
print('the percentages of null values per feature:\n')
print(round(nulls_percentage,2))

In [ ]:
#Is it have no null features so it printed nothing,otherwise it would have printed the percentage of null values per feature.

In [ ]:
#Now deleting the columns which are not required

In [ ]:
del df['ID']
del df["ZIP Code"]
df.head()

In [ ]:
df.groupby('CreditCard').CreditCard.count()

In [ ]:
#It is an imbalanced dataset will see if accuracy decreases.

In [ ]:
df.rename(columns={"Personal Loan": "Personal_Loan","Securities Account":"Sec_Acc","CD Account":"CD_Acc"},inplace=True)
df.head()

In [ ]:
df['Personal_Loan'].value_counts()

In [ ]:
#it is also imbalanced we have to balance the dataset

# EDA

In [ ]:
#Reordering Columns

In [ ]:
# df=df[['Age','Experience','Income',"Family","CCAvg","Education","Mortgage","Sec_Acc","CD_Acc","Online","CreditCard","Personal_Loan"]]

In [ ]:
df.describe()

In [ ]:
df.dtypes

In [ ]:
sns.countplot(data=df,x="Personal_Loan")

In [ ]:
sns.countplot(data=df,x="CreditCard")

In [ ]:
df.tail()

In [ ]:
pip install bubbly

In [ ]:
# from bubbly.bubbly import bubbleplot

In [ ]:
# figure = bubbleplot(dataset = df, x_column = 'Experience', y_column = 'Income', 
#     bubble_column = 'Personal_Loan', time_column = 'Age', size_column = 'Mortgage', color_column = 'Personal_Loan', 
#     x_title = "Experience", y_title = "Income", title = 'Experience vs Income. vs Age vs Mortgage vs Personal Loan',
#     x_logscale = False, scale_bubble = 3, height = 650)

# plt.iplot(figure, config={'scrollzoom': True})

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(df['Age'], color = 'cyan')
plt.title('Distribution of Age', fontsize = 20)
plt.show()

In [ ]:
#Age is well managed

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(df['Income'], color = 'red',norm_hist=True)
plt.title('Distribution of Income', fontsize = 20)
plt.show()

In [ ]:
#So income is not well managed.

In [ ]:
plt.rcParams['figure.figsize'] = (15, 5)
sns.distplot(df['Experience'], color = 'green')
plt.title('Distribution of Experience', fontsize = 20)
plt.show()

In [ ]:
#So Experience is welll balanced

In [ ]:
plt.rcParams['figure.figsize'] = (12, 9)
sns.boxplot(df['Personal_Loan'], df['Income'], palette = 'viridis')
plt.title('Relation of Income with Personal Loan', fontsize = 20)
plt.show()

# FEATURE SELECTION

## CHECKING FOR CORRELATION


In [ ]:
plt.rcParams['figure.figsize'] = (20, 10)
plt.style.use('ggplot')
sns.heatmap(df.corr(), annot = True, cmap = 'Wistia')
plt.title('Heatmap for the Dataset', fontsize = 20)
plt.show()

In [ ]:
corr=df.corr()
corr["Personal_Loan"].sort_values(ascending=False)

In [ ]:
x=df.drop('Personal_Loan',axis=1)
x

In [ ]:
y=df['Personal_Loan']

## Using Extra Trees Classifier

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier
import matplotlib.pyplot as plt
model = ExtraTreesClassifier()
model.fit(x,y)

In [ ]:
print(model.feature_importances_)

In [ ]:
feat_importances = pd.Series(model.feature_importances_, index=x.columns)
feat_importances.nlargest(10).plot(kind='barh')
plt.show()

## Using ANOVA TEST

In [ ]:
from sklearn.feature_selection import f_classif

In [ ]:
select=f_classif(x,y)
select

In [ ]:
p_values=pd.Series(select[1])
p_values.index=x.columns
p_values.sort_values(ascending=True,inplace=True)

In [ ]:
p_values

In [ ]:
p_values=p_values[p_values<0.05]
p_values

In [ ]:
# From all the three feature selection techniques we have got the top features i.e Income,CCAvg,CC_Acc,Mortgage,Education

In [ ]:
#Hence droping less coreleated features.

In [ ]:
df.drop(['Online','CreditCard'],axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
df['Personal_Loan'].value_counts()

In [ ]:
#It seems to be a imbalanced dataset still trying different classification algorithms.

# TRAIN TEST AND SPLIT

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(x,y,test_size=0.20,random_state=0)

In [ ]:
np.bincount(Y_train)

# MODEL FITTING

In [ ]:
# pip install imblearn

In [ ]:
from imblearn.over_sampling import SMOTE

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression

lr1 = LogisticRegression()

lr1.fit(X_train,Y_train)

Y_pred_lr1 = lr1.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
score_lr1 = round(accuracy_score(Y_pred_lr1,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr1)+" %")

In [ ]:
results = confusion_matrix(Y_test, Y_pred_lr1) 

In [ ]:
print(classification_report(Y_pred_lr1,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_lr1)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# TRYING K FOLD CROSS VALIDATION

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
score=cross_val_score(lr1,x,y,cv=10)

In [ ]:
score

In [ ]:
score.mean()

In [ ]:
# So by k fold cross validation we get that our model's accuracy will lie from 93.4% to 96.6% with a mean of 94.7%

In [ ]:
#No we have to oversample the data by using SMOTE ALgorithm as our True negatives are less.Here you can also use Nearmiss(undersampling) and check for the accuracy.

In [ ]:
smt=SMOTE()
x_new,y_new= smt.fit_sample(x, y)

In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(x_new,y_new,test_size=0.20,random_state=0)

In [ ]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()

lr.fit(X_train,Y_train)

Y_pred_lr = lr.predict(X_test)

In [ ]:
np.bincount(Y_train)

In [ ]:
score_lr = round(accuracy_score(Y_pred_lr,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")

In [ ]:
results = confusion_matrix(Y_test, Y_pred_lr) 
results

In [ ]:
print(classification_report(Y_pred_lr,Y_test))

# **HYPER PARAMETER TUNING FOR LOGISTIC REGRESSION MODEL**

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=30)
logreg_cv.fit(X_train,Y_train)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

In [ ]:
logreg2=LogisticRegression(C=1000,penalty="l2")
logreg2.fit(X_train,Y_train)
print("score",logreg2.score(X_test,Y_test))

In [ ]:
Y_pred_lr_2 = logreg2.predict(X_test)

In [ ]:
score_lr = round(accuracy_score(Y_pred_lr_2,Y_test)*100,2)

print("The accuracy score achieved using Logistic Regression is: "+str(score_lr)+" %")

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_lr)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
#Now our true negatives are also more so it now predicting well.

# NAIVE BAYES

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_train,Y_train)

Y_pred_nb = nb.predict(X_test)

In [ ]:
score_nb = round(accuracy_score(Y_pred_nb,Y_test)*100,2)

print("The accuracy score achieved using Naive Bayes is: "+str(score_nb)+" %")

In [ ]:
results = confusion_matrix(Y_test, Y_pred_nb) 
results

In [ ]:
print(classification_report(Y_pred_nb,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_nb)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# K NEAREST NEIGHBORS 

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(X_train,Y_train)
Y_pred_knn=knn.predict(X_test)

In [ ]:
score_knn = round(accuracy_score(Y_pred_knn,Y_test)*100,2)

print("The accuracy score achieved using KNN is: "+str(score_knn)+" %")

In [ ]:
results = confusion_matrix(Y_test, Y_pred_knn) 
results

In [ ]:
print(classification_report(Y_pred_knn,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_knn)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# DECISION TREE

In [ ]:
from sklearn.tree import DecisionTreeClassifier

max_accuracy = 0


for x in range(200):
    dt = DecisionTreeClassifier(random_state=x)
    dt.fit(X_train,Y_train)
    Y_pred_dt = dt.predict(X_test)
    current_accuracy = round(accuracy_score(Y_pred_dt,Y_test)*100,2)
    if(current_accuracy>max_accuracy):
        max_accuracy = current_accuracy
        best_x = x
        


dt = DecisionTreeClassifier(random_state=best_x)
dt.fit(X_train,Y_train)
Y_pred_dt = dt.predict(X_test)

In [ ]:
score_dt = round(accuracy_score(Y_pred_dt,Y_test)*100,2)

print("The accuracy score achieved using Decision Tree is: "+str(score_dt)+" %")

In [ ]:
results = confusion_matrix(Y_test, Y_pred_dt) 
results

In [ ]:
print(classification_report(Y_pred_dt,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_dt)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# RANDOM FOREST

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
model=RandomForestClassifier()

In [ ]:
model.fit(X_train,Y_train)

In [ ]:
Y_pred_rf=model.predict(X_test)

In [ ]:
score_rf = round(accuracy_score(Y_pred_rf,Y_test)*100,2)

print("The accuracy score achieved using Random Forest is: "+str(score_rf)+" %")

In [ ]:
confusion_matrix(Y_test,Y_pred_rf)

In [ ]:
print(classification_report(Y_pred_rf,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_rf)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

# XGBOOST

In [ ]:
import xgboost as xgb

xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=42)
xgb_model.fit(X_train, Y_train)

Y_pred_xgb = xgb_model.predict(X_test)

In [ ]:
score_xgb = round(accuracy_score(Y_pred_xgb,Y_test)*100,2)

print("The accuracy score achieved using XGBoost is: "+str(score_xgb)+" %")

In [ ]:
confusion_matrix(Y_test,Y_pred_xgb)

In [ ]:
print(classification_report(Y_pred_xgb,Y_test))

In [ ]:
fpr, tpr, thresholds = roc_curve(Y_test,Y_pred_xgb)

fig, ax = plt.subplots()
ax.plot(fpr, tpr)
ax.plot([0, 1], [0, 1], transform=ax.transAxes, ls="-", c=".3")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])

plt.rcParams['figure.figsize'] = (5, 5)
plt.title('ROC curve for classifier', fontweight = 30)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
auc = auc(fpr, tpr)
print("AUC Score :", auc)

# NEURAL NETWORK

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential()
model.add(Dense(11,activation='relu',input_dim=11))
model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,epochs=300)

In [ ]:
Y_pred_nn = model.predict(X_test)

In [ ]:
Y_pred_nn.shape

In [ ]:
rounded = [round(x[0]) for x in Y_pred_nn]

Y_pred_nn = rounded

In [ ]:
score_nn = round(accuracy_score(Y_pred_nn,Y_test)*100,2)

print("The accuracy score achieved using Neural Network is: "+str(score_nn)+" %")

In [ ]:
confusion_matrix(Y_test,Y_pred_nn)

In [ ]:
print(classification_report(Y_pred_nn,Y_test))

In [ ]:
scores = [score_lr,score_nb,score_knn,score_dt,score_rf,score_xgb,score_nn]

algorithms = ["Logistic Regression","Naive Bayes","K-Nearest Neighbors","Decision Tree","Random Forest","XGBoost","Neural Network"]    

for i in range(len(algorithms)):
    print("The accuracy score achieved using "+algorithms[i]+" is: "+str(scores[i])+" %")

In [ ]:
sns.barplot(x=algorithms,y=scores)
plt.rcParams['figure.figsize'] = (15, 7)
plt.grid(True)

In [ ]:
from collections import OrderedDict 
dict = {'Logistic Regression':score_lr,'Naive Bayes':score_nb,'K Nearest Neighbors':score_knn,'Decision Tree':score_dt,'Random Forest':score_rf,'XGBoost':score_xgb,'Neural Network':score_nn} 
dict1 = OrderedDict(sorted(dict.items())) 
print(dict1) 

# SO XGBOOST MODEL GAVE US THE HIGHEST ACCURACY..